In [1]:
import pandas as pd

movies = pd.read_csv('ml-25m/movies.csv')

In [2]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [3]:
import re

#Clean the title to leave only the name and year with no speacial characters
def clean_title(title):
    #Search through the title and remove all characters not in the categories assigned
    return re.sub("[^a-zA-Z0-9 ]","", title)

# The name is more important if no year is listed, otherwise the year plays a big factor

In [6]:
#Clean title with no special Characters
movies['clean_title'] = movies['title'].apply(clean_title)
#Clean title with no Year
movies['clean_title_noY'] = movies['clean_title'].str[:-4]

In [7]:
movies

,movieId,title,genres,clean_title,clean_title_noY
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995,Toy Story
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995,Jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995,Grumpier Old Men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995,Waiting to Exhale
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995,Father of the Bride Part II
...,...,...,...,...,...
62418,209157,We (2018),Drama,We 2018,We
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001,Window of the Soul
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018,Bad Poems
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001,A Girl Thing


In [36]:
#Create a term frequency matrix 
from sklearn.feature_extraction.text import TfidfVectorizer

#Check for unigrams and bigrams (one word and two word sequences)
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf_name_year = vectorizer.fit_transform(movies['clean_title'])
tfidf_name = vectorizer.fit_transform(movies['clean_title_noY'])

In [78]:

name = 'Toy Story 2010'

q = vectorizer.transform([name])
print(q.shape)
print('-----')
print(tfidf_name_year.shape)

print(sim)
print(movies['title'][14814])

(1, 119516)
-----
(62423, 170073)
[0.12448077]
Joan Rivers: A Piece of Work (2010)


In [62]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

#this is our search function, it will return the results of the cosine similarity (the distance between) the frequency matrix 
#when compared to our input
def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    #if the input title's last two chracters are numbers, it is likely that a year has been inputted
    #so we will handle the year as part of the name
    if title[-2:].isdigit():
        
        print(title)
        print(query_vec.shape)
        print(tfidf_name_year.shape)
        similarity = cosine_similarity(query_vec, tfidf_name_year).flatten()
    else:
        similarity = cosine_similarity(query_vec, tfidf_name).flatten()
    #argpartition the array such that at the -5th spot we have the largest 5 values in increasing order, then we will slice off only these 5 values
    indeces = np.argpartition(similarity,-5)[-5:]
    #flip the indeces backwards such that the largest value is in the 0th position and descends
    indeces = indeces[::-1]
    #locate the movies corresponding to the indeces
    result = movies.iloc[indeces]
    return result
        

In [63]:
search('Toy Story (2010)')

Toy Story 2010
(1, 119516)
(62423, 170073)


ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 119516 while Y.shape[1] == 170073